# VacationPy: Geoapify API Data
---

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import geoviews as gv
import pandas as pd
import requests
import cartopy.crs as ccrs
from holoviews import dim

# Import API key
from api_keys import geoapify_key

# Enable Bokeh for visualization
gv.extension('bokeh')

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,-22.70,98,83,0.46,GL,1741200360
1,1,atafu village,-8.5421,-172.5159,27.86,70,100,4.09,TK,1741200362
2,2,adamstown,-25.0660,-130.1015,25.65,75,92,5.47,PN,1741200363
3,3,constantia,44.1833,28.6500,5.38,63,48,3.18,RO,1741200088
4,4,mount pearl,47.5166,-52.7813,2.30,83,75,8.75,CA,1741200283


---

### Step 1: Generate a map to display a point for every city in `city_data_df`

In [5]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo=True,  # Enable geographic plotting
    tiles="CartoLight",  # Use a light-colored map like in the screenshot
    size="Humidity",  # Point size based on humidity
    color="City",  # Different colors for each city
    cmap="Category20",  # Use categorical colors to differentiate cities
    alpha=0.7,  # Slight transparency to reduce overlap
    frame_width=1000, 
    frame_height=600,
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down `city_data_df` to locate your ideal weather conditions

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
214,214,kailua-kona,19.6406,-155.9956,24.95,74,0,0.00,US,1741200618
218,218,aswan,24.0934,32.9070,21.61,26,0,0.00,EG,1741200623
297,297,holualoa,19.6228,-155.9522,24.35,73,0,0.45,US,1741200726
389,389,alice springs,-23.7000,133.8833,22.75,15,0,3.60,AU,1741200845
468,468,east london,-33.0153,27.9116,24.52,88,0,2.57,ZA,1741200946
508,508,callao,-12.0667,-77.1500,26.16,65,0,4.12,PE,1741200704
534,534,lima,-12.0432,-77.0282,25.14,65,0,4.12,PE,1741200310


### Step 3: Create a new DataFrame labeled `hotel_df`.

In [9]:
# Create a DataFrame to store city, country, coordinates, and humidity using Pandas copy function
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," for storing hotel names found via Geoapify
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
214,kailua-kona,US,19.6406,-155.9956,74,
218,aswan,EG,24.0934,32.9070,26,
297,holualoa,US,19.6228,-155.9522,73,
389,alice springs,AU,-23.7000,133.8833,15,
468,east london,ZA,-33.0153,27.9116,88,
508,callao,PE,-12.0667,-77.1500,65,
534,lima,PE,-12.0432,-77.0282,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of set coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Extract latitude and longitude
    lat, lng = row["Lat"], row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    hotel_data = response.json()

    # Retrieve the first hotel found and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display data
hotel_df

Starting hotel search...
kailua-kona - nearest hotel: PACIFIC 19 Kona
aswan - nearest hotel: Oscar Hotel
holualoa - nearest hotel: Kona Hotel
alice springs - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
lima - nearest hotel: Hotel Peach Garden


,City,Country,Lat,Lng,Humidity,Hotel Name
214,kailua-kona,US,19.6406,-155.9956,74,PACIFIC 19 Kona
218,aswan,EG,24.0934,32.9070,26,Oscar Hotel
297,holualoa,US,19.6228,-155.9522,73,Kona Hotel
389,alice springs,AU,-23.7000,133.8833,15,No hotel found
468,east london,ZA,-33.0153,27.9116,88,No hotel found
508,callao,PE,-12.0667,-77.1500,65,Casa Ronald
534,lima,PE,-12.0432,-77.0282,65,Hotel Peach Garden


### Step 5: Add hotel name and the country as additional information in the hover message for each city on the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo=True,  # Enable geographic plotting
    tiles="OSM",  # Use OpenStreetMap background
    size=100,  # Fixed size for markers
    color="City",  # Assign distinct colors to each city
    cmap="Category20",  # Use categorical colors for better differentiation
    alpha=0.75,  # Slight transparency
    hover_cols=["City", "Hotel Name", "Humidity", "Country"],  # Show key info on hover
    frame_width=900, 
    frame_height=500, 
    title="Hotel Locations in Ideal Weather Cities"
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Humidity,Country)